# Week 3 of IBM Advanced Data Science Capstone Project

## For this section, I will apply simple predictive modeling using PySpark ML before using the algorithms in Python sci-kit learn library.

### Import csv file from previous week


In [1]:
!wget https://raw.githubusercontent.com/aziz0519/Advanced-Data-Science-Capstone-Project/master/listing_model.csv  

--2020-03-24 06:03:29--  https://raw.githubusercontent.com/aziz0519/Advanced-Data-Science-Capstone-Project/master/listing_model.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 546564 (534K) [text/plain]
Saving to: ‘listing_model.csv’

listing_model.csv   100%[===================>] 533.75K  --.-KB/s    in 0.08s   

2020-03-24 06:03:31 (6.25 MB/s) - ‘listing_model.csv’ saved [546564/546564]



In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
data = pd.read_csv('listing_model.csv')
data.head()

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,distance_to_nearest_stn
0,2,42,1.31100,103.79580,1,4.430817,180,1,0.693147,365,-0.481366
1,0,6,1.33235,103.78521,1,4.382027,90,18,0.000000,365,-0.921634
2,2,42,1.31100,103.79667,1,4.248495,6,20,0.693147,365,-0.645523
3,1,37,1.34541,103.95712,1,5.332719,1,20,2.079442,365,-0.720591
4,1,37,1.34567,103.95963,1,4.553877,1,24,2.079442,365,-0.735902


In [4]:
data.columns

Index(['neighbourhood_group', 'neighbourhood', 'latitude', 'longitude',
       'room_type', 'price', 'minimum_nights', 'number_of_reviews',
       'calculated_host_listings_count', 'availability_365',
       'distance_to_nearest_stn'],
      dtype='object')

In [5]:
data.isnull().sum()

neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
calculated_host_listings_count    0
availability_365                  0
distance_to_nearest_stn           0
dtype: int64

### Training of model

##### Key features of model training processs

* <strong>  Cross Validation </strong> : Using a 10-fold cross validation
* <strong> On each run of cross validation fit 6 models </strong> : Ridge, SVR, Lasso, ElasticNet and Random Forest and Light GBM
* <strong> Stacking </strong> : Train a meta using StackedCVRegreesor optimized used XGboost
* <strong> Blending </strong> : The model trained will overfit the training data hence blending the models together will obtain robust predictions of the Airbnb property prices


In [0]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [0]:
from sklearn.model_selection import KFold , cross_val_score
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale, StandardScaler, RobustScaler
from sklearn.decomposition import PCA

#### Set up cross-validation and define error metrics


In [0]:
kfold = KFold(n_splits=10, random_state=42, shuffle=True)

In [0]:
def rmsle(y, y_pred):
  return np.sqrt(mean_squared_error(y,y_pred))

def rmsecv(model, X=X):
  rmse = np.sqrt(-cross_val_score(model, X, X_train,scoring="neg_mean_squared_error",cv=kfold))

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = data[['neighbourhood_group', 'neighbourhood', 'latitude', 'longitude',
       'room_type', 'minimum_nights', 'number_of_reviews',
       'calculated_host_listings_count', 'availability_365',
       'distance_to_nearest_stn']]
y = data['price']

In [0]:
X_train, X_test , y_train , y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [0]:
#LightGBM
lgb_model = LGBMRegressor(objective='regression',num_leaves=5, learning_rate=0.01,n_estimators=1000,max_bin=200,min_sum_hessian_in_leaf=11,verbose=-1, random_state=42)

In [0]:
#SVR 
svr_model = make_pipeline(RobustScaler(),SVR(C=20,epsilon=0.008,gamma=0.0003))

In [0]:
#Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=1000,max_depth=15,min_samples_split=5,min_samples_leaf=5,max_features=None,oob_score=True,random_state=42)

In [0]:
#ElasticNet Model
eln_model = ElasticNet()

In [0]:
y_pred = lr.predict(X_test)

In [0]:
y_test

8006    168
737     130
3014    101
7512    120
3105     60
       ... 
4845    150
6803    160
3148    600
6187    101
3948    129
Name: price, Length: 2654, dtype: int64

In [0]:
prediction = lr.predict(X_test)
error_airbnb = pd.DataFrame({
        'Actual Values': np.array(y_test).flatten(),
        'Predicted Values': prediction.flatten()})
error_airbnb.head()

,Actual Values,Predicted Values
0,168,171.345585
1,130,190.093650
2,101,167.762828
3,120,254.822666
4,60,213.719478
